# Simple Example

Before running the algorithm, please install the package first. For installation detail please check `README.rst` in the root directory.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from phre import ImageData, measure_data
from phre.optimizer import find_initial_image, RelaxSplit

## Load data

In [ ]:
image = np.load('../data/mnist_digit.npy')
data = ImageData(image)

In [ ]:
data.show_image()

## Create observations

In [ ]:
obs_mat, obs = measure_data(data,
                            num_obs=data.image_size*4,
                            obs_std=0.0,
                            normalize_obs_mat=True)

## Initialize variables

In [ ]:
init_x = find_initial_image(obs_mat, obs)
init_w = obs_mat.dot(init_x)

In [ ]:
plt.imshow(init_x.reshape(data.image_shape))

## Create solver and optimize the problem

In [ ]:
solver_rs = RelaxSplit(obs_mat, obs, nu=1.0)

In [ ]:
result_rs = solver_rs.phase_retrieval(init_x=init_x,
                                      init_w=init_w,
                                      max_iter=500,
                                      verbose=True)

## Final result

In [ ]:
plt.imshow(result_rs.reshape(data.image_shape))